In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd
import demoji
from langdetect import detect
import re
import os

In [2]:
CLIENT_SECRET_FILE = 'client_secret.json'

In [3]:
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICES_NAME = 'youtube'
API_VERSION = 'v3'

In [4]:
def get_authenticated_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE,SCOPES)
    credentials = flow.run_console()
    return build(API_SERVICES_NAME,API_VERSION,credentials = credentials)
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
services = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=400862598816-3v31cqlk7igcgo3es6o5kiq6l7hs2a64.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=M8Q9UpBkNDLWLGWXn1fztOJ2JYHTwx&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g4ptD19-45ee-7MLOuwmYGWlf6qTFE38tuFpccZ25pSKOeHPGh_d2c


In [17]:
query = 'Bojhena Shey Bojhena | Title Track | Soham | Abir | Payel | Mimi | Arijit Singh | Indraadip'

In [18]:
query_results = services.search().list(part = 'snippet',q = query, order = 'relevance',
                                       type='video',relevanceLanguage = 'en',
                                       safeSearch = 'moderate').execute()

video_id = []
channel = []
video_title = []
video_desc = []

for item in query_results['items']:
    video_id.append(item['id']['videoId'])
    channel.append(item['snippet']['channelTitle'])
    video_title.append(item['snippet']['title'])
    video_desc.append(item['snippet']['description'])
    
video_id = video_id[0]
channel = channel[0]
video_title = video_title[0]
video_desc = video_desc[0]

video_id_pop = []
channel_pop = []
video_title_pop = []
video_desc_pop = []
comments_pop = []
comment_id_pop = []
reply_count_pop = []
like_count_pop = []

comments_temp = []
comment_id_temp = []
reply_count_temp = []
like_count_temp = []

nextPage_token = None

while 1:
    response = services.commentThreads().list(part='snippet',
                                            videoId = video_id,
                                            maxResults = 100,
                                            order = 'relevance',
                                            textFormat = 'plainText',
                                            pageToken = nextPage_token).execute()
    nextPage_token = response.get('nextPageToken')
    for item in response['items']:
        comments_temp.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
        comment_id_temp.append(item['snippet']['topLevelComment']['id'])
        reply_count_temp.append(item['snippet']['totalReplyCount'])
        like_count_temp.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
        comments_pop.extend(comments_temp)
        comment_id_pop.extend(comment_id_temp)
        reply_count_pop.extend(reply_count_temp)
        like_count_pop.extend(like_count_temp)        
        video_id_pop.extend([video_id]*len(comments_temp))
        channel_pop.extend([channel]*len(comments_temp))
        video_title_pop.extend([video_title]*len(comments_temp))
        video_desc_pop.extend([video_desc]*len(comments_temp))
        
    if nextPage_token is None:
        break
        
output_dict = {
    'channel' : channel_pop,
    'video title' : video_title_pop,
    'video Description' : video_desc_pop,
    'video ID' : video_id_pop,
    'comment' : comments_pop,
    'commnet ID' : comment_id_pop,
    'Replies' : reply_count_pop,
    'Likes' : like_count_pop,
}

df = pd.DataFrame(output_dict,columns = output_dict.keys())

duplicate_comments = df[df.duplicated('comment')]
print('number of duplicate comments: ',duplicate_comments.shape[0])
print('number of unique comments: ',df.shape[0]-duplicate_comments.shape[0])
unique_comments = df.drop_duplicates(subset=['comment'])

number of duplicate comments:  1999067
number of unique comments:  1933


In [19]:
unique_comments.to_csv('dataset.csv',index = False)